In [1]:
from torch_geometric.data import download_url, extract_zip
import pandas as pd

dataset_name = 'ml-latest-small'

url = f'https://files.grouplens.org/datasets/movielens/{dataset_name}.zip'
extract_zip(download_url(url, '.'), '.')

movies_path = f'./{dataset_name}/movies.csv'
ratings_path = f'./{dataset_name}/ratings.csv'

Using existing file ml-latest-small.zip
Extracting .\ml-latest-small.zip


In [2]:
rating_df = pd.read_csv(ratings_path)
movies_df = pd.read_csv(movies_path)

In [29]:
userId = 'userId'
mappedUserId = 'mappedUserId'
movieId = 'movieId'
mappedMovieId = 'mappedMovieId'
user= 'user'
movie = 'movie'
rates = 'rates'
rev_rates = 'rev_rates'

In [4]:
unique_user_id = rating_df[userId].unique()
unique_user_id = pd.DataFrame(data= {
    userId :unique_user_id,
    mappedUserId : pd.RangeIndex(len(unique_user_id)) 
})

In [5]:
unique_movie_id = movies_df[movieId].unique()
unique_movie_id = pd.DataFrame(data={
    movieId : unique_movie_id,
    mappedMovieId : pd.RangeIndex(len(unique_movie_id))
})

In [6]:
rating_df = rating_df.merge(unique_user_id, on=userId)
rating_df = rating_df.merge(unique_movie_id, on=movieId)
rating_df

,userId,movieId,rating,timestamp,mappedUserId,mappedMovieId
0,1,1,4.0,964982703,0,0
1,5,1,4.0,847434962,4,0
2,7,1,4.5,1106635946,6,0
3,15,1,2.5,1510577970,14,0
4,17,1,4.5,1305696483,16,0
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,609,9325
100832,610,160527,4.5,1479544998,609,9330
100833,610,160836,3.0,1493844794,609,9342
100834,610,163937,3.5,1493848789,609,9389


In [7]:
import torch

In [8]:
edge_index = torch.stack([
    torch.tensor(rating_df[mappedUserId].values),
    torch.tensor(rating_df[mappedMovieId].values)
])
edge_index

tensor([[   0,    4,    6,  ...,  609,  609,  609],
        [   0,    0,    0,  ..., 9342, 9389, 9390]])

In [9]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [12]:
genres = movies_df['genres'].str.get_dummies('|').values
genres = torch.from_numpy(genres).to(torch.float)
genres

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
from sentence_transformers import SentenceTransformer

In [15]:
model = SentenceTransformer('all-MiniLM-L6-v2')
with torch.no_grad():
    titles = model.encode(movies_df['title'].tolist(), convert_to_tensor=True, show_progress_bar=True)
    titles = titles.cpu()

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

In [16]:
titles

tensor([[-0.0828,  0.0530,  0.0536,  ...,  0.0226,  0.0538,  0.1030],
        [-0.1053,  0.1508, -0.0264,  ...,  0.0106, -0.0726,  0.0086],
        [-0.0988,  0.0176, -0.0527,  ..., -0.0120,  0.0303,  0.0004],
        ...,
        [-0.1115,  0.0310, -0.0177,  ...,  0.0147,  0.0299,  0.0200],
        [ 0.0366,  0.0137,  0.0315,  ..., -0.0516, -0.0143,  0.1012],
        [-0.0500, -0.0141, -0.0031,  ...,  0.0320,  0.0546, -0.0271]])

In [23]:
movie_features = torch.cat([genres, titles], dim = -1)
movie_features.size()

torch.Size([9742, 404])

In [22]:
user_feature = torch.eye(len(unique_user_id))
user_feature.size()

torch.Size([610, 610])

In [24]:
import torch_geometric.transforms as T
from torch_geometric.data import HeteroData

In [25]:
data = HeteroData()

In [28]:
data[user].x = user_feature
data[movie].x = movie_features
data[user,rates,movie].edge_index = edge_index
data[user,rates,movie].edge_lable = torch.from_numpy(rating_df['rating'].values).to(torch.float)
data = T.ToUndirected()(data)

In [30]:
data[movie, rev_rates, user]

{'edge_index': tensor([[   0,    0,    0,  ..., 9342, 9389, 9390],
        [   0,    4,    6,  ...,  609,  609,  609]]), 'edge_lable': tensor([4.0000, 4.0000, 4.5000,  ..., 3.0000, 3.5000, 3.5000])}

In [31]:
data

HeteroData(
  user={ x=[610, 610] },
  movie={ x=[9742, 404] },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_lable=[100836],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 100836],
    edge_lable=[100836],
  }
)

In [33]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[(user, rates, movie)],
    rev_edge_types= [(movie, rev_rates, user)]
)(data)

train_data, val_data

(HeteroData(
   user={ x=[610, 610] },
   movie={ x=[9742, 404] },
   (user, rates, movie)={
     edge_index=[2, 80670],
     edge_lable=[80670],
     edge_label=[80670],
     edge_label_index=[2, 80670],
   },
   (movie, rev_rates, user)={
     edge_index=[2, 80670],
     edge_lable=[80670],
   }
 ),
 HeteroData(
   user={ x=[610, 610] },
   movie={ x=[9742, 404] },
   (user, rates, movie)={
     edge_index=[2, 80670],
     edge_lable=[80670],
     edge_label=[10083],
     edge_label_index=[2, 10083],
   },
   (movie, rev_rates, user)={
     edge_index=[2, 80670],
     edge_lable=[80670],
   }
 ))